<a href="https://colab.research.google.com/github/nathalierocelle/CapstoneProject_GamAuth/blob/main/GamAuth_yolov5_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLOv5 Image Classification

## Setup and Import Useful Libraries

In [2]:
!git clone https://github.com/ultralytics/yolov5 
%cd yolov5
%pip install -qr requirements.txt

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5


In [3]:
!pip install roboflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import os
import torch
import utils
from roboflow import Roboflow
display = utils.notebook_init()

YOLOv5 🚀 v7.0-53-g65071da Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 23.0/78.2 GB disk)


## Load the dataset


In [5]:
os.makedirs("../datasets/", exist_ok=True)
%cd ../datasets/

/content/datasets


In [6]:
rf = Roboflow(api_key="1f25N6kvuPDRURqSNfHf")
project = rf.workspace("deep-learning-kpf72").project("medicine-authenticity-classification-foqrv")
dataset = project.version(1).download("folder")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to medicine-authenticity-classification-1 in folder:: 100%|██████████| 637/637 [00:00<00:00, 794.09it/s]


In [7]:
dataset_name = dataset.location.split(os.sep)[-1]
os.environ["DATASET_NAME"] = dataset_name

In [8]:
dataset_name

'medicine-authenticity-classification-1'

## Model Training

In [9]:
%cd ../yolov5
!python classify/train.py --model yolov5s-cls.pt --data $DATASET_NAME --epochs 100 --img 128 --pretrained weights/yolov5s-cls.pt

/content/yolov5
classify/train: model=yolov5s-cls.pt, data=medicine-authenticity-classification-1, epochs=100, batch_size=64, imgsz=128, nosave=False, cache=None, device=, workers=8, project=runs/train-cls, name=exp, exist_ok=False, pretrained=weights/yolov5s-cls.pt, optimizer=Adam, lr0=0.001, decay=5e-05, label_smoothing=0.1, cutoff=None, dropout=None, verbose=False, seed=0, local_rank=-1
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-53-g65071da Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

TensorBoard: Start with 'tensorboard --logdir runs/train-cls', view at http://localhost:6006/
albumentations: RandomResizedCrop(p=1.0, height=128, width=128, scale=(0.08, 1.0), ratio=(0.75, 1.3333333333333333), interpolation=1), HorizontalFlip(p=0.5), ColorJitter(p=0.5, brightness=[0.6, 1.4], contrast=[0.6, 1.4], saturation=[0.6, 1.4], hue=[0, 0]), Normalize(p=1.0, mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0), ToTensor

## Model Validation

In [10]:
!python classify/val.py --weights runs/train-cls/exp/weights/best.pt --data ../datasets/$DATASET_NAME

classify/val: data=../datasets/medicine-authenticity-classification-1, weights=['runs/train-cls/exp/weights/best.pt'], batch_size=128, imgsz=224, device=, workers=8, verbose=True, project=runs/val-cls, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-53-g65071da Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 117 layers, 4174374 parameters, 0 gradients, 10.4 GFLOPs
testing: 100% 1/1 [00:00<00:00,  3.30it/s]
                   Class      Images    top1_acc    top5_acc
                     all          26       0.846           1
        bioflu_authentic           5           1           1
      bioflu_counterfeit           3       0.667           1
      biogesic_authentic           6       0.833           1
    biogesic_counterfeit           4        0.75           1
        neozep_authentic           5           1           1
      neozep_counterfeit           3       0.667           1
Speed: 0.1ms pre-process, 4.3ms inference

 ## Model Testing

In [11]:
if os.path.exists(os.path.join(dataset.location, "test")):
  split_path = os.path.join(dataset.location, "test")
else:
  os.path.join(dataset.location, "valid")
example_class = os.listdir(split_path)[0]
example_image_name = os.listdir(os.path.join(split_path, example_class))[0]
example_image_path = os.path.join(split_path, example_class, example_image_name)
os.environ["TEST_IMAGE_PATH"] = example_image_path

print(f"Inferring on an example of the class '{example_class}'")

#Infer
!python classify/predict.py --weights runs/train-cls/exp/weights/best.pt --source $TEST_IMAGE_PATH

Inferring on an example of the class 'neozep_counterfeit'
classify/predict: weights=['runs/train-cls/exp/weights/best.pt'], source=/content/datasets/medicine-authenticity-classification-1/test/neozep_counterfeit/image_94_png.rf.b6c9925fa2cc659ad79dc8df30342573.jpg, data=data/coco128.yaml, imgsz=[224, 224], device=, view_img=False, save_txt=False, nosave=False, augment=False, visualize=False, update=False, project=runs/predict-cls, name=exp, exist_ok=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-53-g65071da Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 117 layers, 4174374 parameters, 0 gradients, 10.4 GFLOPs
image 1/1 /content/datasets/medicine-authenticity-classification-1/test/neozep_counterfeit/image_94_png.rf.b6c9925fa2cc659ad79dc8df30342573.jpg: 224x224 neozep_counterfeit 0.90, neozep_authentic 0.05, biogesic_counterfeit 0.03, bioflu_authentic 0.01, bioflu_counterfeit 0.00, 3.9ms
Speed: 0.3ms pre-process, 3.9ms inference, 

In [13]:
os.environ["TEST_CLASS_PATH"] = test_class_path = os.path.join(*os.environ["TEST_IMAGE_PATH"].split(os.sep)[:-1])
print(f"Infering on all images from the directory {os.environ['TEST_CLASS_PATH']}")

!python classify/predict.py --weights runs/train-cls/exp/weights/best.pt --source '/content/datasets/medicine-authenticity-classification-1/test/*/*.jpg'

Infering on all images from the directory content/datasets/medicine-authenticity-classification-1/test/neozep_counterfeit
classify/predict: weights=['runs/train-cls/exp/weights/best.pt'], source=/content/datasets/medicine-authenticity-classification-1/test/*/*.jpg, data=data/coco128.yaml, imgsz=[224, 224], device=, view_img=False, save_txt=False, nosave=False, augment=False, visualize=False, update=False, project=runs/predict-cls, name=exp, exist_ok=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-53-g65071da Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 117 layers, 4174374 parameters, 0 gradients, 10.4 GFLOPs
image 1/26 /content/datasets/medicine-authenticity-classification-1/test/bioflu_authentic/image_135_png.rf.47b7e076028461ac8062dac37bd2a037.jpg: 224x224 bioflu_authentic 0.99, biogesic_authentic 0.00, neozep_authentic 0.00, neozep_counterfeit 0.00, bioflu_counterfeit 0.00, 4.4ms
image 2/26 /content/datasets/medicine-authent

### Testing several input methods
- Webcam: `python classify/predict.py --weights yolov5s-cls.pt --source 0`
- Image `python classify/predict.py --weights yolov5s-cls.pt --source img.jpg`
- Video: `python classify/predict.py --weights yolov5s-cls.pt --source vid.mp4`
- Directory: `python classify/predict.py --weights yolov5s-cls.pt --source path/`
- Glob: `python classify/predict.py --weights yolov5s-cls.pt --source 'path/*.jpg'`
- YouTube: `python classify/predict.py --weights yolov5s-cls.pt --source 'https://youtu.be/Zgi9g1ksQHc'`
- RTSP, RTMP, HTTP stream: `python classify/predict.py --weights yolov5s-cls.pt --source 'rtsp://example.com/media.mp4'`